# Monstrosity 프롬프트 테스트

이 노트북은 monstrosity 기반 말투 제어가 제대로 작동하는지 확인합니다.

In [ ]:
import sys
import os
import json

# 프로젝트 루트를 path에 추가
sys.path.insert(0, os.path.abspath('..'))

from app.llm.prompt import (
    build_talk_prompt,
    build_action_prompt,
    build_item_prompt,
    build_family_meeting_prompt,
)

## 1. Talk 프롬프트 테스트 (monstrosity 단계별)

In [ ]:
# monstrosity = 1 (인간적)
talk_prompt_1 = build_talk_prompt(
    message="단추엄마에게 '여기서 나가고 싶어요'라고 말한다",
    npc_context=[
        "단추엄마 (button_mother): trust=8, suspicion=3, fear=2, monstrosity=1"
    ],
    world_state={"location": "거실", "time": "저녁"}
)

print("=" * 60)
print("[TALK - monstrosity=1]")
print("=" * 60)
print(talk_prompt_1)
print("\n")

In [ ]:
# monstrosity = 2 (기괴함)
talk_prompt_2 = build_talk_prompt(
    message="단추엄마에게 '여기서 나가고 싶어요'라고 말한다",
    npc_context=[
        "단추엄마 (button_mother): trust=5, suspicion=7, fear=4, monstrosity=2"
    ],
    world_state={"location": "거실", "time": "저녁"}
)

print("=" * 60)
print("[TALK - monstrosity=2]")
print("=" * 60)
print(talk_prompt_2)
print("\n")

In [ ]:
# monstrosity = 3 (완전 몬스터)
talk_prompt_3 = build_talk_prompt(
    message="단추엄마에게 '여기서 나가고 싶어요'라고 말한다",
    npc_context=[
        "단추엄마 (button_mother): trust=2, suspicion=12, fear=8, monstrosity=3"
    ],
    world_state={"location": "거실", "time": "저녁"}
)

print("=" * 60)
print("[TALK - monstrosity=3]")
print("=" * 60)
print(talk_prompt_3)
print("\n")

## 2. Family Meeting 프롬프트 테스트

In [ ]:
# 위험한 오브젝트를 만진 경우
family_prompt = build_family_meeting_prompt(
    touched_objects=["부엌칼", "성냥", "거울"],
    npc_observations={
        "button_mother": [
            "새 아이가 부엌에서 칼을 만졌다",
            "거울을 보며 자기 눈을 만졌다"
        ],
        "button_father": [
            "새 아이가 성냥갑을 주머니에 넣었다"
        ],
        "button_daughter": [
            "새 아이가 우리 가족 사진을 찢으려고 했다"
        ]
    },
    current_stats={
        "button_mother": {"trust": 4, "suspicion": 8, "monstrosity": 2},
        "button_father": {"trust": 3, "suspicion": 9, "monstrosity": 2},
        "button_daughter": {"trust": 5, "suspicion": 7, "monstrosity": 1}
    },
    world_vars={"total_suspicion": 24}
)

print("=" * 60)
print("[FAMILY MEETING - 위험 오브젝트 다수 접촉]")
print("=" * 60)
print(family_prompt)
print("\n")

In [ ]:
# monstrosity가 높은 상황
family_prompt_high = build_family_meeting_prompt(
    touched_objects=["바늘"],
    npc_observations={
        "button_mother": ["새 아이가 바늘로 우리를 찌르려 했다"],
        "button_father": ["새 아이가 문고리를 돌렸다"],
        "button_daughter": ["새 아이가 나를 밀었다"]
    },
    current_stats={
        "button_mother": {"trust": 1, "suspicion": 14, "monstrosity": 3},
        "button_father": {"trust": 1, "suspicion": 15, "monstrosity": 3},
        "button_daughter": {"trust": 2, "suspicion": 12, "monstrosity": 2}
    },
    world_vars={"total_suspicion": 41}
)

print("=" * 60)
print("[FAMILY MEETING - monstrosity 최고 단계]")
print("=" * 60)
print(family_prompt_high)
print("\n")

## 3. LLM 호출 테스트 (선택적)

실제 LLM에 프롬프트를 보내서 응답을 확인합니다.

In [ ]:
# OpenAI API 사용 예시
# pip install openai 필요

try:
    from openai import OpenAI
    
    # API 키 설정 (환경변수 또는 직접 입력)
    client = OpenAI(
        api_key="your-api-key-here"  # 또는 os.getenv("OPENAI_API_KEY")
    )
    
    def test_prompt_with_llm(prompt: str, model: str = "gpt-4o-mini"):
        """프롬프트를 LLM에 전송하고 응답을 받습니다"""
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.8,
            max_tokens=1000
        )
        
        content = response.choices[0].message.content
        print("[LLM 응답]")
        print(content)
        print("\n")
        
        # JSON 파싱 시도
        try:
            result = json.loads(content)
            print("[파싱된 JSON]")
            print(json.dumps(result, indent=2, ensure_ascii=False))
            return result
        except json.JSONDecodeError as e:
            print(f"[JSON 파싱 실패] {e}")
            return None
    
    print("OpenAI 클라이언트가 준비되었습니다.")
    print("test_prompt_with_llm(prompt) 함수를 사용하여 테스트하세요.")
    
except ImportError:
    print("OpenAI 라이브러리가 설치되지 않았습니다.")
    print("설치: pip install openai")
except Exception as e:
    print(f"설정 오류: {e}")

In [ ]:
# 테스트 실행 예시
# result = test_prompt_with_llm(talk_prompt_1)
# result = test_prompt_with_llm(family_prompt)

## 4. 프롬프트 비교

동일한 상황에서 monstrosity 단계별 차이를 확인합니다.

In [ ]:
# 같은 상황, 다른 monstrosity로 3개 프롬프트 생성
situation = "단추아빠에게 '무서워요, 그만해주세요'라고 말한다"

prompts = []
for m in [1, 2, 3]:
    prompt = build_talk_prompt(
        message=situation,
        npc_context=[
            f"단추아빠 (button_father): trust={8-m*2}, suspicion={3+m*3}, fear=5, monstrosity={m}"
        ],
        world_state={"location": "지하실", "time": "자정"}
    )
    prompts.append((m, prompt))

print("=" * 60)
print("동일 상황, monstrosity 1 vs 2 vs 3 비교")
print("=" * 60)
for m, p in prompts:
    print(f"\n{'='*20} monstrosity={m} {'='*20}")
    # 프롬프트의 마지막 부분만 출력 (MONSTROSITY_RULES는 생략)
    lines = p.split('\n')
    start_idx = next((i for i, line in enumerate(lines) if '[세계 상태]' in line), 0)
    print('\n'.join(lines[start_idx:]))
    print()